In [9]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from typing import List

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer.add_tokens(["<ROOT>", "<EMPTY>"], special_tokens=True)

print(device)
torch.manual_seed(99)

cpu


 # Data


In [10]:
from utils import is_projective, generate_gold_pathmoves 

def match_subtokens(l1:List[str], l2:List[str]):
    # Create output list
    output:List[List[int]] = []
    # Initialize index for l2
    index = 0
    # Iterate through l1
    for token in l1:
        subtoken_indices = []
        # Get the indices of the subtokens
        while index < len(l2) and (not subtoken_indices or l2[index].startswith("#")):
            subtoken_indices.append(index)
            index += 1
        # Append subtoken indices to output
        output.append(subtoken_indices)
    return output

def merge_splitted_tokens(tokens:List[str]):
    out:List[str]=[]
    len=0
    for t in tokens:
      if t[0]=="#":
        out[len-1]+=t.strip("#")
      else:
        out.append(t)
        len+=1
    return out


def tokens_tokenizer_correspondence(tokens:List[List[str]], berttokens:List[List[int]]):
    global tokenizer
    correspondences:List[List[List[int]]]=[]
    
    for t,bt in zip(tokens, berttokens):
        correspondences.append(match_subtokens(t, list(tokenizer.convert_ids_to_tokens(bt))))
    return correspondences


def get_configurations(toks:List[List[str]], heads:List[List[int]], get_gold_path=False):
  '''
  toks: list of list of tokens
  heads: list of list of heads
  '''
  # put sentence and gold tree in our format
      # gold_path and gold_moves are parallel arrays whose elements refer to parsing steps
  gold_configurations:List[List[List[int]]]= (
      []
  )  # record two topmost stack tokens and first 2 buffer token for current step
  gold_moves:List[List[int]] = (
      []
  )  # contains oracle (canonical) move for current step: 0 is left, 1 right, 2 shift, 3 reduce
  gold_heads:List[List[int]]=[]
  
  for tokens, head in zip(toks, heads):
      conf=[]   
      mov=[] 
      tokens = ["<ROOT>"] + tokens
      head = [-1] + list(map(int,head))

      if get_gold_path:  # only for training
          conf, mov=generate_gold_pathmoves(tokens, head)
          
          
      gold_configurations.append(conf)
      gold_moves.append(mov)
      gold_heads.append(head)
  print(f'gold_configurations = {gold_configurations} \n, gold_moves = {gold_moves}\n, gold_heads = {gold_heads}\n\n')
  input()
  return gold_configurations, gold_moves,gold_heads
  

def prepare_batch(batch_data,get_gold_path=False):
    '''
    :param batch_data: batch from dataloader
    :param get_gold_path: if True, return gold path and moves
    
    :return: tokenizer()
    :return: configurations
    :return: moves
    :return: heads
    :return: correspondences
    '''
    # Extract embeddings
    tok_sentences= tokenizer(
        ["<ROOT> "+ bd["text"] for bd in batch_data],
        padding=True, 
        return_tensors="pt",
        add_special_tokens=False 
    )

    # get gold path and moves
    configurations, moves, head = get_configurations(
        [bd["tokens"] for bd in batch_data],
        [bd["head"] for bd in batch_data],
        get_gold_path
    )
    
    # get correspondences between tokens and bert tokens (subword)
    correspondences=tokens_tokenizer_correspondence(
        [["<ROOT>"]+bd["tokens"] for bd in batch_data],
        tok_sentences["input_ids"]  # type: ignore 
    )
    print(f'tok_sentences = {tok_sentences} \n, configurations = {configurations}\n, moves = {moves}\n, head = {head}\n, correspondences = {correspondences}\n\n')
    return tok_sentences, configurations, moves, head, correspondences

In [11]:
from datasets import load_dataset

train_dataset=load_dataset("universal_dependencies", "en_lines", split="train")
validation_dataset=load_dataset("universal_dependencies", "en_lines", split="validation")
test_dataset=load_dataset("universal_dependencies", "en_lines", split="test")
print(
    f"train_dataset: {len(train_dataset)}, validation_dataset: {len(validation_dataset)}, test_dataset: {len(test_dataset)}" #type:ignore
) 


# remove non projective
train_dataset = train_dataset.filter(lambda x:is_projective([-1]+list(map(int,x['head'])))) 
validation_dataset = validation_dataset.filter(lambda x:is_projective([-1]+list(map(int,x['head']))))
test_dataset = test_dataset.filter(lambda x:is_projective([-1]+list(map(int,x['head']))))
print(
    f"PROJECTIVE -> train_dataset: {len(train_dataset)}, validation_dataset: {len(validation_dataset)}, test_dataset: {len(test_dataset)}" #type:ignore
)  

Found cached dataset universal_dependencies (/home/piermarco/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)
Found cached dataset universal_dependencies (/home/piermarco/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)
Found cached dataset universal_dependencies (/home/piermarco/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)
Loading cached processed dataset at /home/piermarco/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7/cache-b982dab38278edb0.arrow
Loading cached processed dataset at /home/piermarco/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7/cache-f0ce6ef98dbfeebb.arr

train_dataset: 3176, validation_dataset: 1032, test_dataset: 1035
PROJECTIVE -> train_dataset: 2922, validation_dataset: 930, test_dataset: 968


 # NET


In [12]:
BATCH_SIZE = 1
DIM_CONFIG = 2
LSTM_ISBI = True
BERT_SIZE = 768
EMBEDDING_SIZE = BERT_SIZE
LSTM_LAYERS = 1
MLP_SIZE = 200
CLASSES = 4
DROPOUT = 0.2
EPOCHS = 20 # 30
LR = 0.001  # learning rate
NUM_LABELS_OUT = 4

 ## Dataloader

In [13]:
train_dataloader = torch.utils.data.DataLoader( # type:ignore
  train_dataset,
  batch_size=BATCH_SIZE, 
  shuffle=True,
  collate_fn=lambda x: prepare_batch(x, get_gold_path=True)
)

validation_dataloader = torch.utils.data.DataLoader( # type: ignore
  validation_dataset,
  batch_size=BATCH_SIZE,
  shuffle=True,
  collate_fn=lambda x: prepare_batch(x, get_gold_path=True)
)

test_dataloader = torch.utils.data.DataLoader( # type:ignore
  test_dataset,
  batch_size=BATCH_SIZE,
  shuffle=True,
  collate_fn=lambda x: prepare_batch(x, get_gold_path=False)
)

In [14]:
from transformers import AutoModel
from arceagerparser import ArcEager, Oracle
from utils import parse_step

class BERTNet(nn.Module):
  def __init__(self,device) -> None:
    super().__init__()
    self.device=device
    
    self.embeddings = nn.Embedding(
        len(tokenizer), EMBEDDING_SIZE, padding_idx=0
    )
    
    self.bert = AutoModel.from_pretrained('bert-base-uncased')
    self.bert.resize_token_embeddings(len(tokenizer))
    for param in self.bert.parameters():
      param.requires_grad = False
    
    self.w1=nn.Linear(DIM_CONFIG*BERT_SIZE, 3*MLP_SIZE)
    self.w2=nn.Linear(3*MLP_SIZE, CLASSES)
    self.activation= nn.Tanh()
    self.softmax=nn.Softmax(dim=-1)
    self.dropout = nn.Dropout(DROPOUT)
  

  def getAvgH(self, h, corr):
    avgH=torch.zeros(BERT_SIZE,requires_grad=False).to(self.device)
    for i in corr:
        avgH+=h[i]
        avgH/=len(corr)
    return avgH
  
  def get_mlp_input(self, configs, h, correspondences):
    mlp_input=[]
    zero_tensor=torch.zeros(BERT_SIZE,requires_grad=False, device=self.device)
    for i, (conf, corr) in enumerate(zip(configs, correspondences)):
      for j in conf:
        mlp_input.append(
          torch.cat([
            zero_tensor if j[0]==-1 else self.getAvgH(h[i], corr[j[0]]),
            zero_tensor if j[1]==-1 else self.getAvgH(h[i], corr[j[1]])
          ])
        )
    mlp_input=torch.stack(mlp_input).to(self.device)
    return mlp_input
  
  def mlp_pass(self, x):
      return self.softmax(
          self.w2(self.dropout(self.activation(self.w1(self.dropout(x)))))
      )

  
  def forward(self, bertInput, configs, correspondencens):
    bertInput=bertInput.to(self.device)
    input_ids=bertInput['input_ids'].to(self.device)
    attention_mask=bertInput['attention_mask'].to(self.device)
    
    h = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state.to(self.device)

    mlp_input=self.get_mlp_input(configs, h, correspondencens)
    out=self.mlp_pass(mlp_input)

    return out

  
  def get_configurations(self, parsers):
    configurations = []
    for parser in parsers:
      if parser.is_tree_final():
        conf = [-1, -1]
      else:
        conf = [
          parser.stack[len(parser.stack) - 1],
        ]
        if len(parser.buffer) == 0:
          conf.append(-1)
        else:
          conf.append(parser.buffer[0])
      configurations.append([conf])
    # print(f"configurations {configurations}")
    return configurations

  
  def infere(self, bertInput):
    bertInput=bertInput.to(self.device)
    input_ids=bertInput['input_ids'].to(self.device)
    attention_mask=bertInput['attention_mask'].to(self.device)
    merged_tokens=[merge_splitted_tokens(list(tokenizer.convert_ids_to_tokens(tok))) for tok in input_ids]

    parsers:List[ArcEager] =[ArcEager(tok) for tok in merged_tokens]
    correspondences= tokens_tokenizer_correspondence(merged_tokens , input_ids)
    h = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state.to(self.device)
    
    
    while not all([t.is_tree_final() for t in parsers]):
      # get the current configuration and score next moves
      configurations = self.get_configurations(parsers)
      mlp_input = self.get_mlp_input(configurations, h, correspondences).to(self.device)
      mlp_out = self.mlp_pass(mlp_input).to(self.device)
      # take the next parsing step
      parse_step(parsers, mlp_out)

    # return the predicted dependency tree
    return [parser.arcs for parser in parsers]
  
      
model = BERTNet(device).to(device)
print(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTNet(
  (embeddings): Embedding(30524, 768, padding_idx=0)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30524, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((7

 ## Run the model

In [15]:
from utils import evaluate

def train(model:BERTNet, dataloader:torch.utils.data.DataLoader, criterion, optimizer): #type:ignore
    model.train()  # setup model for training mode
    total_loss = 0
    count = 0
    
    for batch in dataloader:
        print(f"TRAINING: batch {count}/{len(dataloader):.0f}")
        optimizer.zero_grad()
        sentences, paths, moves, _, correspondences = batch

        out = model(sentences, paths, correspondences)

        labels = torch.tensor(sum(moves, [])).to(
            device
        )  # sum(moves, []) flatten the array

        loss = criterion(out, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        count += 1

    return total_loss / count


def test(model:BERTNet, dataloader:torch.utils.data.DataLoader): #type:ignore
    model.eval()
    
    gold = []
    preds = []
    count = 0
    for batch in dataloader:
        print(f"TEST: batch {count}/{len(dataloader):.0f}")
        sentences, _ , _ , head, _ = batch
        
        with torch.no_grad():
            pred = model.infere(sentences )
            gold += head
            preds += pred
            count += 1

    return evaluate(gold, preds)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

with open("bert.log", "w") as f:
    for epoch in range(EPOCHS):
        print("Starting Epoch", epoch)
        avg_train_loss = train(model, train_dataloader, criterion, optimizer)
        torch.save(model.state_dict(), f"bert_e{epoch+1}.pt")
        #avg_train_loss = -1
        #torch.load(f"model_e{epoch}.pt")
        val_uas = test(model, validation_dataloader)

        log= f"Epoch: {epoch:3d} | avg_train_loss: {avg_train_loss:.5f} | dev_uas: {val_uas:.5f} |\n"
        print(log)
        f.write(log)

        #save the model on pytorch format

    test_uas = test(model, test_dataloader)
    log = f"test_uas: {test_uas:5.3f}"
    print(log)
    f.write(log + "\n")

Starting Epoch 0


KeyboardInterrupt: 